<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/AI_agents/Langchain_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Libraries

In [1]:
!pip install langchain langchain-openai openai wikipedia langchain-community langchain-experimental -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
OpenAI_API = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OpenAI_API

## React Agent with tools

A Zero-shot ReAct Agent refers to a type of agent used in ReAct (Reasoning and Acting) frameworks that can perform tasks without requiring specific training or fine-tuning on task-specific examples. In the zero-shot setting, the agent is capable of reasoning and acting on a given task or prompt without being explicitly trained on that exact task.

In [3]:
import pandas as pd
import os
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_community.chat_models import ChatOpenAI
from langchain_core.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain import hub

In [4]:
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

<ipython-input-4-4faf22c6b3bb>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")


In [5]:
# Load the required tools (Wikipedia and LLM Math) to be used by the agent
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
# Initialize the agent with the tools and set the agent type to ZERO_SHOT_REACT_DESCRIPTION
# The 'verbose=True' argument enables detailed logging during execution
agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
result_1 = agent.run("I am going to buy 5 boxes of apple. Each box containes 2 apples. If I don't buy 5 boxes of apple instead buy 3 and get 2 boxes of oranges which contain 3 oranges per box. How many total apples will I have?")
# Print the result of the agent's response to the query
print(result_1)

<ipython-input-5-3156c10a1a1e>:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools , llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
<ipython-input-5-3156c10a1a1e>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result_1 = agent.run("I am going to buy 5 boxes 



> Entering new AgentExecutor chain...
To find out how many total apples I will have, I need to calculate the number of apples in the 3 boxes I plan to buy. Each box contains 2 apples, so I will multiply the number of boxes by the number of apples per box. 

Action: Calculator
Action Input: 3 * 2
Observation: Answer: 6
Thought:I have calculated that if I buy 3 boxes of apples, I will have 6 apples. Now, I need to consider the oranges, but since the question only asks for the total number of apples, I don't need to calculate the oranges. 

Final Answer: 6 apples.

> Finished chain.
6 apples.


In [ ]:
result_2 = agent.run("Who is the highest mountain peaks of world? What are the other two highest mountain peaks in the world following the first?")
print(result_2)



> Entering new AgentExecutor chain...
To answer the question about the highest mountain peaks in the world, I need to look up information regarding the tallest mountains. The highest peak is known to be Mount Everest, but I need to confirm this and find the next two highest peaks as well. 

Action: wikipedia  
Action Input: "highest mountain peaks in the world"  
Observation: Page: List of highest mountains on Earth
Summary: There are at least 108 mountains on Earth with elevations of 7,200 m (23,600 ft; 4.5 mi) or greater above sea level. Of these, 14 are more than 8,000 m (26,000 ft; 5.0 mi). The vast majority of these mountains are located on the edge of the Indian Plate and Eurasian Plate in China, India, Nepal, and Pakistan.

The dividing line between a mountain with multiple peaks and separate mountains is not always clear (see also Highest unclimbed mountain). A popular and intuitive way to distinguish mountains from subsidiary peaks is by their height above the highest saddle

## Add memory to Agent

In [7]:
from langchain.memory import ConversationBufferMemory

In [8]:
# Initialize memory to store the conversation history with a key 'chat_history'
memory = ConversationBufferMemory(memory_key="chat_history")  # Storing the conversation history

# Initialize the conversational agent with specified configurations
conversational_agent = initialize_agent(
    agent='conversational-react-description',  # Set the agent type to conversational-react-description
    tools=tools,  # Provide the tools (e.g., Wikipedia, LLM math) to the agent
    llm=llm,  # Assign the language model to the agent
    verbose=True,  # Enable detailed logging of the agent's actions
    max_iterations=3,  # Limit the number of iterations for the agent's response
    memory=memory,  # Pass the conversation memory to store the chat history
)

<ipython-input-8-62ebcf29c8e9>:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")  # Storing the conversation history


In [9]:
result_3 = conversational_agent.run("Who won the 2022 Fifa world cup?")
print(result_3)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: 2022 FIFA World Cup winner
Observation: Page: 2022 FIFA World Cup
Summary: The 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Middle East and Persian Gulf countries, and the second in an Asian country after the 2002 tournament in South Korea and Japan.
This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot and humid climate in summers, the event was held in November and December, becoming the first one to take place outside the traditional months of May, June, and July. It was held over a reduced time frame of 29 days with 

In [16]:
result_3=conversational_agent.run("Which teams were in top 10?")
print(result_3)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: wikipedia
Action Input: 2022 FIFA World Cup final standings
Observation: Page: 2022 FIFA World Cup
Summary: The 2022 FIFA World Cup was the 22nd FIFA World Cup, the quadrennial world championship for national football teams organized by FIFA. It took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Middle East and Persian Gulf countries, and the second in an Asian country after the 2002 tournament in South Korea and Japan.
This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot and humid climate in summers, the event was held in November and December, becoming the first one to take place outside the traditional months of May, June, and July. It was held over a reduced time frame of 29 d

## Custom Tool

In [11]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [12]:
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
 """Adds two numbers together"""
 return a + b # the actions our tool performs

In [15]:
# Create a chat prompt template with structured messages
# The template defines the system behavior and placeholders for dynamic values
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
          You are a mathematical assistant. Use your tools to answer questions.
          If you do not have a tool to answer the question, say so.

          Return only the answers. e.g
          Human: What is 1 + 1?
          AI: 2
          """),  # Defining the system's role and expected behavior
        MessagesPlaceholder("chat_history", optional=True),  # Optional placeholder for the conversation history
        ("human", "{input}"),  # Placeholder for the human's input
        MessagesPlaceholder("agent_scratchpad"),  # Placeholder for the agent's internal work or scratchpad
    ]
)

# Define the toolkit that includes the 'add' operation (used for mathematical operations)
toolkit = [add]

# Create an agent using the OpenAI model with the defined prompt and toolkit
agent = create_openai_tools_agent(llm, toolkit, prompt)

# Create an agent executor to handle the agent's execution and verbose logging
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

# Run the agent executor with an input query and print the result
result = agent_executor.invoke({"input": "what is 2  + 2"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 2, 'b': 2}`


44

> Finished chain.
{'input': 'what is 2  + 2', 'output': '4'}
